In [4]:
import pandas as pd

## Функция чтения xml

In [5]:
def read_xml(data_raw):
    
    namespaces_codes = 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure'
    regiones_codes = pd.read_xml(data_raw,
                                xpath="//structure:Code",
                                namespaces = {"structure": namespaces_codes})
    regiones_codes['value'] = regiones_codes['value'].astype(str)

    namespaces_values = 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic'

    regiones_values = pd.read_xml(
        data_raw, xpath="//generic:Value", 
        namespaces = {"generic": namespaces_values}
        )
    
    concept_list = regiones_values['concept'].drop_duplicates() # Забираю список концептов и удалаю дубликаты

    regiones_values_pivot = pd.DataFrame() # Новая пустая таблица
    
    for i in concept_list:
        regiones_values_pivot[i] = regiones_values.loc[regiones_values['concept'] == i]['value'].values

    regiones_Obs = pd.read_xml(
        data_raw, 
        xpath="//generic:Obs", 
        namespaces = {"generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic"}
    )

    regiones_ObsValue = pd.read_xml(data_raw, xpath="//generic:ObsValue", namespaces = {"generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic"})

    regiones_ObsValue2 = regiones_ObsValue
    regiones_ObsValue2 ['value2'] = regiones_ObsValue['value'].astype(str).str.replace(',','.').replace('.',None).astype(float)

    data = pd.concat([
                regiones_values_pivot, # Первая таблица с данными значений
                regiones_Obs['Time'], # Столбец из таблицы со значениями года
                regiones_ObsValue2['value2'] # Столбец из таблицы статистических значений
                ],
                axis=1 # Указываем, что столбцы мы объединяем по горизонтали
            ).merge(regiones_codes, # Добавляем таблицу со списком регионов
                    left_on='s_OKATO', # Указываем столбец по котоорму будет вестись консолидация в левой исходной таблице
                    right_on='value', # Указываем в каком столбце лежат данные для консолидации в правой присоединяемой таблице
            ).drop(columns='value') # Удаляем столбец из правой таблицы, который дублирует столбец левой таблицы

    return data

In [8]:
data_raw='Число зарегистрированных разводов (оперативные данные).xml'
read_xml(data_raw)

,s_OKATO,EI,PERIOD,Time,value2,Description
0,643,единица,январь,2006,41381.0,Российская Федерация
1,643,единица,февраль,2006,45510.0,Российская Федерация
2,643,единица,январь-февраль,2006,86891.0,Российская Федерация
3,643,единица,март,2006,59581.0,Российская Федерация
4,643,единица,январь-март,2006,146472.0,Российская Федерация
...,...,...,...,...,...,...
30046,45001000000,единица,январь-октябрь,2013,36171.0,Москва в старых границах
30047,45001000000,единица,ноябрь,2013,4060.0,Москва в старых границах
30048,45001000000,единица,январь-ноябрь,2013,40231.0,Москва в старых границах
30049,45001000000,единица,декабрь,2013,4052.0,Москва в старых границах
